1. Menentukan Library apa saja yang akan digunakan

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

2. Load Dataset

In [331]:
training_dataset = pd.read_excel('dataKasus-1.xlsx')

In [332]:
#menampilkan 5 baris awal data
training_dataset.head()


,NO,NAMA,USIA,PARITAS,JARAK KELAHIRAN,RIW HIPERTENSI,RIW PE,OBESITAS,RIW DM,RIW HIPERTENSI/PE DALAM KELUARGA,SOSEK RENDAH,PE/Non PE
0,1,NAMA 1,23 TH,3,< 2 tahun,Tidak,Tidak,Tidak,Ya,Tidak,>UMR,Non PE
1,2,NAMA 2,29 TH,2,< 2 tahun,Tidak,PEB,Tidak,Tidak,Ada,>UMR,PE
2,3,NAMA 3,20 TH,1,anak pertama,Tidak,Tidak,Tidak,Tidak,Tidak,>UMR,Non PE
3,4,NAMA 4,18 TH,1,anak pertama,Tidak,Tidak,Tidak,Tidak,Tidak,>UMR,Non PE
4,5,NAMA 5,34 TH,3,> 2 tahun,Tidak,Tidak,Tidak,Tidak,Tidak,>UMR,Non PE


In [333]:
#menampi9lkan jumlah (baris,kolom)
training_dataset.shape

(561, 12)

In [334]:
#menghitung jumlah PE/Non PE
training_dataset['PE/Non PE'].value_counts()

PE/Non PE
Non PE                    520
PEB                        20
PE                         17
Eklamsia                    1
PE gemelli                  1
PEB impending eklampsi      1
PE                          1
Name: count, dtype: int64

In [335]:
#strip values to float type
df = pd.DataFrame(training_dataset)
df['PE/Non PE'] = df['PE/Non PE'].map({'PE': 1.0, 'Non PE': 0.0, 'PEB':0.0})

In [336]:
print(df.head())

   NO    NAMA   USIA  PARITAS JARAK KELAHIRAN RIW HIPERTENSI RIW PE OBESITAS  \
0   1  NAMA 1  23 TH        3       < 2 tahun          Tidak  Tidak    Tidak   
1   2  NAMA 2  29 TH        2       < 2 tahun          Tidak    PEB    Tidak   
2   3  NAMA 3  20 TH        1    anak pertama          Tidak  Tidak    Tidak   
3   4  NAMA 4  18 TH        1    anak pertama          Tidak  Tidak    Tidak   
4   5  NAMA 5  34 TH        3      > 2 tahun           Tidak  Tidak    Tidak   

  RIW DM RIW HIPERTENSI/PE DALAM KELUARGA SOSEK RENDAH  PE/Non PE  
0     Ya                            Tidak         >UMR        0.0  
1  Tidak                              Ada         >UMR        1.0  
2  Tidak                            Tidak         >UMR        0.0  
3  Tidak                            Tidak         >UMR        0.0  
4  Tidak                            Tidak         >UMR        0.0  


In [353]:
#menampilkan data hasilnya
df

,NO,NAMA,USIA,PARITAS,JARAK KELAHIRAN,RIW HIPERTENSI,RIW PE,OBESITAS,RIW DM,RIW HIPERTENSI/PE DALAM KELUARGA,SOSEK RENDAH,PE/Non PE
0,1,1.0,23.0,3,2,0,2,0,1,0,1,0.0
1,2,2.0,29.0,2,2,0,2,0,0,1,1,1.0
2,3,3.0,20.0,1,1,0,2,0,0,0,1,0.0
3,4,4.0,18.0,1,1,0,2,0,0,0,1,0.0
4,5,5.0,34.0,3,3,0,2,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
556,558,557.0,40.0,3,3,0,2,0,0,0,1,0.0
557,559,558.0,28.0,3,2,0,2,0,0,0,1,0.0
558,560,559.0,41.0,3,3,0,2,0,0,0,1,0.0
559,561,560.0,32.0,2,3,0,2,0,0,0,1,0.0


In [339]:
#convert string menjadi float

df['NAMA'] = df['NAMA'].astype(str)
df['NAMA'] = df['NAMA'].str.replace('NAMA ', '').astype(float)
df['USIA'] = df['USIA'].astype(str)
df['USIA'] = df['USIA'].str.replace(' TH', '').str.replace('TH', '').str.replace(' th', '').str.replace('th', '').astype(float)

df['JARAK KELAHIRAN'] = df['JARAK KELAHIRAN'].replace({'anak pertama': 1, '< 2 tahun': 2, '> 2 tahun': 3})
df['RIW HIPERTENSI'] = df['RIW HIPERTENSI'].replace({'Tidak': 0, 'Ya': 1})
df['RIW PE'] = df['RIW PE'].replace({'Tidak': 0, 'PEB': 1})
df['OBESITAS'] = df['OBESITAS'].replace({'Tidak': 0, 'Ya': 1})
df['RIW DM'] = df['RIW DM'].replace({'Ya': 1, 'Tidak': 0})
df['RIW HIPERTENSI/PE DALAM KELUARGA'] = df['RIW HIPERTENSI/PE DALAM KELUARGA'].replace({'Tidak': 0, 'Ada': 1})
df['SOSEK RENDAH'] = df['SOSEK RENDAH'].replace({'>UMR': 1, '<UMR': 0})

# JARAK KELAHIRAN
def convert_jarak_kelahiran(x):
    x = str(x).strip()
    if x == 'anak pertama':
        return 1
    elif x == '< 2 tahun':
        return 2
    elif x == '> 2 tahun':
        return 3
    else:
        return x

df['JARAK KELAHIRAN'] = df['JARAK KELAHIRAN'].apply(convert_jarak_kelahiran)

# RIW HIPERTENSI
def convert_riw_hipertensi(x):
    x = str(x).strip()
    if x == 'Tidak':
        return 0
    elif x == 'Ya':
        return 1
    else:
        return x

df['RIW HIPERTENSI'] = df['RIW HIPERTENSI'].apply(convert_riw_hipertensi)

# RIW PE
def convert_riw_pe(x):
    x = str(x).strip()
    if x == 'Tidak':
        return 0
    elif x == 'PEB':
        return 1
    else:
        return 2

df['RIW PE'] = df['RIW PE'].apply(convert_riw_pe)

# OBESITAS
def convert_obesitas(x):
    x = str(x).strip()
    if x == 'Tidak':
        return 0
    elif x == 'Ya':
        return 1
    else:
        return x

df['OBESITAS'] = df['OBESITAS'].apply(convert_obesitas)

# RIW DM
def convert_riw_dm(x):
    x = str(x).strip()
    if x == 'Ya':
        return 1
    elif x == 'Tidak':
        return 0
    else:
        return x

df['RIW DM'] = df['RIW DM'].apply(convert_riw_dm)

# RIW HIPERTENSI/PE DALAM KELUARGA
def convert_riw_hipertensi_pe_dalam_keluarga(x):
    x = str(x).strip()
    if x == 'Tidak':
        return 0
    elif x == 'Ada':
        return 1
    else:
        return x

df['RIW HIPERTENSI/PE DALAM KELUARGA'] = df['RIW HIPERTENSI/PE DALAM KELUARGA'].apply(convert_riw_hipertensi_pe_dalam_keluarga)

# SOSEK RENDAH
def convert_sosek_rendah(x):
    x = str(x).strip()
    if x == '>UMR':
        return 1
    elif x == '<UMR':
        return 0
    else:
        return x

df['SOSEK RENDAH'] = df['SOSEK RENDAH'].apply(convert_sosek_rendah)

In [340]:
#memisahkan data dan label
X = df.drop (columns='PE/Non PE', axis=1)
Y = df['PE/Non PE']

In [341]:
#cek X
print(X)

      NO   NAMA  USIA  PARITAS JARAK KELAHIRAN RIW HIPERTENSI  RIW PE  \
0      1    1.0  23.0        3               2              0       2   
1      2    2.0  29.0        2               2              0       2   
2      3    3.0  20.0        1               1              0       2   
3      4    4.0  18.0        1               1              0       2   
4      5    5.0  34.0        3               3              0       2   
..   ...    ...   ...      ...             ...            ...     ...   
556  558  557.0  40.0        3               3              0       2   
557  559  558.0  28.0        3               2              0       2   
558  560  559.0  41.0        3               3              0       2   
559  561  560.0  32.0        2               3              0       2   
560  562  561.0  30.0        2               3              0       2   

    OBESITAS RIW DM RIW HIPERTENSI/PE DALAM KELUARGA SOSEK RENDAH  
0          0      1                                0   

In [342]:
#cek Y
print(Y)

0      0.0
1      1.0
2      0.0
3      0.0
4      0.0
      ... 
556    0.0
557    0.0
558    0.0
559    0.0
560    0.0
Name: PE/Non PE, Length: 561, dtype: float64


3. Standarisasi Data

In [346]:
#test run scaller
scaler = StandardScaler()

In [347]:
scaler.fit(X)

StandardScaler()

In [348]:
standarized_data = scaler.transform(X)

In [364]:
print(standarized_data)

[[-1.73202765 -1.72896612 -0.87083235 ... 10.54514106 -0.14142136
   0.08474272]
 [-1.72586076 -1.72279124  0.14291824 ... -0.09483041  7.07106781
   0.08474272]
 [-1.71969387 -1.71661636 -1.37770764 ... -0.09483041 -0.14142136
   0.08474272]
 ...
 [ 1.71526382  1.71661636  2.17041943 ... -0.09483041 -0.14142136
   0.08474272]
 [ 1.72143071  1.72279124  0.64979354 ... -0.09483041 -0.14142136
   0.08474272]
 [ 1.7275976   1.72896612  0.31187668 ... -0.09483041 -0.14142136
   0.08474272]]


In [365]:
X = standarized_data

Y = df['PE/Non PE']
Y = Y.fillna(Y.mean())  # or Y.median()

In [366]:
print(X)
print(Y)

[[-1.73202765 -1.72896612 -0.87083235 ... 10.54514106 -0.14142136
   0.08474272]
 [-1.72586076 -1.72279124  0.14291824 ... -0.09483041  7.07106781
   0.08474272]
 [-1.71969387 -1.71661636 -1.37770764 ... -0.09483041 -0.14142136
   0.08474272]
 ...
 [ 1.71526382  1.71661636  2.17041943 ... -0.09483041 -0.14142136
   0.08474272]
 [ 1.72143071  1.72279124  0.64979354 ... -0.09483041 -0.14142136
   0.08474272]
 [ 1.7275976   1.72896612  0.31187668 ... -0.09483041 -0.14142136
   0.08474272]]
0      0.0
1      1.0
2      0.0
3      0.0
4      0.0
      ... 
556    0.0
557    0.0
558    0.0
559    0.0
560    0.0
Name: PE/Non PE, Length: 561, dtype: float64


4. Memisahkan data training dan data testing

In [367]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, stratify=Y, random_state=2)


In [368]:
#membagi data training (448), data testing(113) 
print(X.shape, X_train.shape, X_test.shape)

(561, 11) (448, 11) (113, 11)


5. Coba membuat data latih menggunakan algoritm SVM apakah possible

In [369]:
classifier = svm.SVC(kernel='linear')

In [374]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier

imputer = SimpleImputer(strategy='mean')
# classifier = SVC()  # Replace with one of the following
classifier = SVR()  # or KNeighborsClassifier()

pipeline = Pipeline([
    ('imputer', imputer),
    ('classifier', classifier)
])

pipeline.fit(X_train, Y_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('classifier', SVR())])

6. Coba membuat model evaluasi untuk ukur tingkat akurasinya

In [379]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

imputer = SimpleImputer(strategy='mean')
classifier = SVR()

pipeline = Pipeline([
    ('imputer', imputer),
    ('classifier', classifier)
])

pipeline.fit(X_train, Y_train)  # Fit the pipeline to the data
X_train_prediction = pipeline.predict(X_train)

# Use mean squared error or mean absolute error to evaluate the performance
training_data_mse = mean_squared_error(Y_train, X_train_prediction)
training_data_mae = mean_absolute_error(Y_train, X_train_prediction)

print("Mean Squared Error:", training_data_mse)
print("Mean Absolute Error:", training_data_mae)

Mean Squared Error: 0.026212978047270963
Mean Absolute Error: 0.10130199731651908


7. Coba membuat Model Prediksi

In [383]:
# Import necessary libraries
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
import numpy as np

# Define the input data with the same number of features as X_train
input_data = np.array([[23.0, 3, 2, 0, 2, 0, 1, 0, 1, 0, 0]])  # Add two more features to match X_train

# Define the pipeline with imputer and SVR model
imputer = SimpleImputer(strategy='mean')
model = SVR()

pipeline = Pipeline([
    ('imputer', imputer),
    ('model', model)
])

# Fit the pipeline to the training data (X_train and Y_train)
pipeline.fit(X_train, Y_train)

# Make predictions on the input data
prediction = pipeline.predict(input_data)

# Round the prediction to the nearest integer (0 or 1)
prediction = round(prediction[0])

# Print the result
if prediction == 1:
    print("PE")
else:
    print("NON PE")

NON PE


8. Simpan

In [384]:
import pickle

In [385]:
filename='training_data.myob'
pickle.dump(classifier, open(filename,'wb'))